In [2]:
import sys
import time
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs

#分配GPU資源
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [3]:
yolo = YoloV3(classes=80)
yolo.load_weights('./checkpoints/yolov3.tf').expect_partial()
print('weights loaded')
class_names = [c.strip() for c in open('./data/coco.names').readlines()]
print('classes loaded')

weights loaded
classes loaded


In [10]:
for i in range(1500):
    os.mkdir("./dataset_cropped/"+str(i).zfill(4))

In [43]:
def get_person_box(img,boxes,nums):
    #尋找最大的行人框並回傳
    person_boxes=[]
    person_boxes_size=[]
    wh = np.flip(img.shape[0:2])
    #找出所有的行人框
    for i in range(nums[0]):
        if classes[0][i]!=0:
            continue
        x1,x2=(boxes[0][i][0]*wh[0]),(boxes[0][i][2]*wh[0])
        y1,y2=(boxes[0][i][1]*wh[1]),(boxes[0][i][3]*wh[1])
        person_boxes.append([x1,y1,x2,y2])
    if len(person_boxes)==1:
        return person_boxes[0]
    if len(person_boxes)==0:
        return person_boxes
    #找出最大行人框
    for box in person_boxes:
        person_boxes_size.append((box[2]-box[0])*(box[3]-box[1]))
    max_idx=np.where(max(person_boxes_size))
    return person_boxes[max_idx[0][0]]
    

In [44]:
def crop_img(img,box,file):
    box=np.array(box).astype(int)
    file=file.split("\r")[0]
    file=file.split("\\")
    person_img=img[box[1]:box[3],box[0]:box[2]]
    person_img=cv2.resize(person_img,(64,128),interpolation = cv2.INTER_AREA)
    cv2.imwrite("dataset_cropped/"+file[1]+"/"+file[2],person_img)

In [46]:
import cv2
for root, dirs, files in os.walk("dataset", topdown=False):
    for name in files:
        #使用os.walk遍歷所有檔案
        file=os.path.join(root, name)
        print(file)
        img_raw = tf.image.decode_image(
            open(file, 'rb').read(), channels=3)
        #進行resize
        img = tf.expand_dims(img_raw, 0)
        img = transform_images(img, 416)

        t1 = time.time()
        boxes, scores, classes, nums = yolo(img)
        if nums[0]==0:
            continue
        t2 = time.time()
        print(t2-t1)
        img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
        box=get_person_box(img,boxes,nums)
        #找到非人的類別
        if len(box)!=4:
            continue
        crop_img(img,box,file)

T0000F0012791.jpg
0.13709568977355957
dataset\1472\1472c1T0000F0012792.jpg
0.12469124794006348
dataset\1472\1472c3T0000F0012793.jpg
dataset\1472\1472c3T0000F0012794.jpg
0.1553642749786377
dataset\1472\1472c3T0000F0012795.jpg
0.13973140716552734
dataset\1472\1472c3T0000F0012796.jpg
0.1351470947265625
dataset\1472\1472c5T0000F0012797.jpg
dataset\1472\1472c5T0000F0012798.jpg
dataset\1472\1472c6T0000F0012799.jpg
dataset\1472\1472c6T0000F0012800.jpg
dataset\1472\1472c6T0000F0012801.jpg
dataset\1473\1473c1T0000F0012772.jpg
dataset\1473\1473c3T0000F0012773.jpg
dataset\1473\1473c3T0000F0012774.jpg
0.13679957389831543
dataset\1473\1473c3T0000F0012775.jpg
dataset\1473\1473c5T0000F0012776.jpg
dataset\1473\1473c5T0000F0012777.jpg
dataset\1473\1473c6T0000F0012778.jpg
dataset\1473\1473c6T0000F0012779.jpg
dataset\1473\1473c6T0000F0012780.jpg
dataset\1473\1473c6T0000F0012781.jpg
dataset\1474\1474c1T0000F0012782.jpg
dataset\1474\1474c1T0000F0012783.jpg
0.13999128341674805
dataset\1474\1474c2T0000F00127

In [17]:
boxes

<tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
array([[[0.061378  , 0.06517133, 0.9057858 , 0.9527755 ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        